In [ ]:
# Refer to https://pythonawesome.com/overview-of-the-peaks-dectection-algorithms-available-in-python/  for finding peaks

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd

In [14]:
df = pd.read_excel('sample_data/20191129_A028300.xlsx')

In [15]:

def smooth(x, window_len=11, window='hanning'):
    
    if x.ndim != 1:
        raise ValueError('only accept 1 dim')
    
    if x.size < window_len:
        raise ValueError('x len too short')

    if window_len < 3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")

    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y
    

In [16]:
df['avg'] = df['close_price']#.rolling(20).mean()

In [17]:
from scipy.signal import argrelextrema

In [18]:
minima = argrelextrema(np.nan_to_num(df['avg'].to_numpy()), np.greater, order=10)

In [19]:
minima

(array([  2,  43, 149, 335, 362]),)

In [20]:
df['avg'].iloc[80:90]

80    120800
81    120500
82    119600
83    119500
84    119500
85    119900
86    119200
87    119100
88    119300
89    119300
Name: avg, dtype: int64

In [21]:
import plotly.graph_objects as go

In [22]:
df['minima'] = False

In [23]:
df['minima'].iloc[minima] = True

/home/users/yongwoo1.park/workspace/python3-env/lib/python3.5/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [28]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=df['date'], y=df['avg'], mode='lines'))
fig.add_trace(go.Scatter(x=df['date'], y=df['close_price'], mode='lines'))
fig.add_trace(go.Scatter(x=df['date'], y=df['cum_buy_volume'], name='culmulate buy', line=dict(color='red')), row=2, col=1)
fig.add_trace(go.Scatter(x=df['date'], y=df['cum_sell_volume'], name='culmulate sell', line=dict(color='blue')), row=2, col=1)
d = df[df['minima'] == True]['date']
shapes = []
for date in d:
    shapes.append(dict(x0=date, x1=date, y0=0, y1=1, xref='x', yref='paper', line_width=2))
fig.update_layout(yaxis_tickformat='d', shapes=shapes)

In [25]:
d = df[df['minima'] == True]['date']
for aa in d:
    print(aa)

2019-11-29 09:03:00
2019-11-29 09:44:00
2019-11-29 11:30:00
2019-11-29 14:36:00
2019-11-29 15:03:00
